In [1]:
%matplotlib notebook
from matplotlib import rc
rc('text', usetex=True)
import matplotlib.pyplot as plt

In [18]:
from PyQNLPSimulator import PyQNLPSimulator as p
import QNLP as q
import numpy as np
from itertools import product

In [19]:
vsm = q.VectorSpaceModel.VectorSpaceModel(
    corpus_path="/Users/mlxd/Desktop/qs_dev/intel-qnlp/corpus/84-0.txt", mode='l', stop_words=False
)

In [4]:
basis     = vsm.define_basis({'verbs' : 16, 'nouns' : 16})

In [5]:
verb_dist = vsm.sort_basis_tokens_by_dist("verbs")
noun_dist = vsm.sort_basis_tokens_by_dist("nouns")

In [12]:
# Define basis tokens encoding and decoding dicts
encoding_dict = {"ns" : vsm.encoded_tokens["nouns"],
                 "v"  : vsm.encoded_tokens["verbs"],
                 "no" : vsm.encoded_tokens["nouns"]
                }

decoding_dict = {"ns" : { v:k for k,v in encoding_dict["ns"].items() },
                 "v"  : { v:k for k,v in encoding_dict["v"].items() },
                 "no" : { v:k for k,v in encoding_dict["no"].items() }
                }

In [22]:
# Register must be large enough to support 2*|nouns| + |verbs|
len_reg_memory = len(verb_dist) + 2*len(noun_dist)
len_reg_ancilla = len_reg_memory + 2
num_qubits = len_reg_memory + len_reg_ancilla
print("Requires {} qubits to encode data".format(num_qubits))

Requires 14 qubits to encode data


In [23]:
"""Require analysis of corpus to determine number of patterns to encode. The current 
encoding method assumes unique patterns, though this can be extended to multiple 
occurrences of the same patterns later. Additionally, we can store 2^num_qubits 
unique patterns. As such, we will restrict the patterns to encode by limiting the
space over which we analyse the corpus-to-basis mapping.
"""
num_bin_pattern = 8